In [1]:
import os

In [2]:
from langchain.tools.retriever import create_retriever_tool

In [7]:
from langchain.agents import AgentExecutor
from langchain.chains import LLMChain
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_ollama import OllamaLLM
from langchain import hub
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate

# Initialize LangChain components
llm = OllamaLLM(model="llama3.2:latest")
prompt = ChatPromptTemplate.from_template("{input}")

# Define chains
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Initialize tools
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
arxiv_tool = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

# Initialize retriever tool
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
db = FAISS.from_documents(documents, embeddings)
db_ret = db.as_retriever()
retrival_tool = create_retriever_tool(db_ret, "langsmith_search","Ask Me Anything about langsmith")

# Combine tools
tools = [wiki_tool, arxiv_tool, retrival_tool]

# Create agent
agent_executor = AgentExecutor(agent=llm_chain, tools=tools, verbose=True)

# Execute agent
input_message = {"user":"Tell me about Langsmith"}
response = agent_executor.invoke(input_message)
print(response)



> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'input'}

In [10]:
from langchain.agents import initialize_agent, Tool
from langchain.chains import LLMChain
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_ollama import OllamaLLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate

# Initialize LangChain components
llm = OllamaLLM(model="llama3.2:latest")
prompt = ChatPromptTemplate.from_template("{input}")

# Define chains
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Initialize tools
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)

# Define tools for the agent
wiki_tool = Tool(
    name="Wikipedia",
    func=api_wrapper.run,
    description="Useful for searching Wikipedia."
)

arxiv_tool = Tool(
    name="Arxiv",
    func=api_wrapper_arxiv.run,
    description="Useful for searching Arxiv papers."
)

# Initialize retriever tool
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
db = FAISS.from_documents(documents, embeddings)
db_ret = db.as_retriever()

# Define retriever tool
retrieval_tool = Tool(
    name="Langsmith_Search",
    func=db_ret.get_relevant_documents,
    description="Retrieves information from Langsmith documentation."
)

# Combine tools
tools = [wiki_tool, arxiv_tool, retrieval_tool]

# Create agent using initialize_agent
agent = initialize_agent(
    tools=tools,
    agent="zero-shot-react-description",
    llm=llm,
    verbose=True
)

# Execute agent with the correct input
input_message = "Tracing to LangSmith for LangChain users"
response = agent.run(input_message)

# Print the response
print(response)




> Entering new AgentExecutor chain...
Question: What is Langsmith?
Thought: I need to find information about Langsmith in order to understand its usage with LangChain.
Action: Langsmith_Search
Action Input: 'Langsmith
Observation: [Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'), Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and